# Person Re-identification using PySDK
This notebook demonstrates Person Re-Identification (Re-ID) using PySDK. Re-ID focuses on recognizing and matching people across different camera views based on their unique appearance, like clothing and body shape.

The basic pipeline works like this:
1. Detect people in the image using a person detection model.

2. Crop each detected person using the bounding box coordinates.

3. Apply the Person Re-ID model to the cropped images to extract the embeddings which can further be used to identify and match individuals across different images or camera views.

In [ ]:
import degirum as dg, degirum_tools

inference_host_address = "@local"
zoo_url = "degirum/hailo"
token = '' 
device_type = "HAILORT/HAILO8L"

# Person detection model name 
person_det_model_name = "yolov8n_relu6_person--640x640_quant_hailort_hailo8l_1"

# choose image source
image_source = "../assets/ThreePersons.jpg"

# load AI model
person_det_model = dg.load_model(
    model_name=person_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
    
)
# Run the inference on the image and see results
print(f" Running inference using '{person_det_model_name}' on image source '{image_source}'")
person_detections = person_det_model(image_source)
print(person_detections)

print("Press 'x' or 'q' to stop.")

# show results of inference
with degirum_tools.Display("AI Camera") as output_display:
    output_display.show_image(person_detections.image_overlay)

### Visualizing Person crops

In [ ]:
# Utility function to visualize the crops
import matplotlib.pyplot as plt
def display_images(images, title="Images", figsize=(15, 5)):
    """
    Display a list of images in a single row using Matplotlib.

    Parameters:
    - images (list): List of images (NumPy arrays) to display.
    - title (str): Title for the plot.
    - figsize (tuple): Size of the figure.
    """
    num_images = len(images)
    fig, axes = plt.subplots(1, num_images, figsize=figsize)
    if num_images == 1:
        axes = [axes]  # Make it iterable for a single image
    for ax, image in zip(axes, images):
        image_rgb = image[:, :, ::-1]  # Convert BGR to RGB
        ax.imshow(image_rgb)
        ax.axis('off')
    fig.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# List to store the person crops
crops = []

# Process each detection result
for person in person_detections.results:
    # Extract bounding box (assumed in [x1, y1, x2, y2] format)
    x1, y1, x2, y2 = map(int, person["bbox"])  # Convert bbox coordinates to integers
    person_crop = person_detections.image[y1:y2, x1:x2]  # Crop the person from the image
    crops.append(person_crop)

# Display person crops
display_images(crops, title="Person Crops", figsize=(5, 3))

### Extracting embedding using a Person recognition model for each person crop

In [ ]:
# Choose the Person reid model name 
person_reid_model_name = "osnet_x1_0_person_reid--256x128_quant_hailort_hailo8l_1"
# person_reid_model_name = "repvgg_a0_person_reid--256x128_quant_hailort_hailo8l_1" 

# load AI model
person_reid_model = dg.load_model(
    model_name=person_reid_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
    
)
# Run the embedding model on each person crop to extract the embeddings of size 512
for person_crop in crops:
    person_embedding = person_reid_model(person_crop).results[0]["data"][0][0] # shape (1,512)
    print (person_embedding.shape)   